In [1]:
# test_hyperclova_response.py
import os
import sys
sys.path.append('.')

from langchain_naver import ChatClovaX
from config import Config
import re
import json

def test_hyperclova_response():
    """HyperClova X 응답 테스트 및 분석"""
    
    # HyperClova X 초기화
    try:
        client = ChatClovaX(
            api_key=Config.HYPERCLOVA_X_API_KEY,
            model=Config.HYPERCLOVA_MODEL,
            max_tokens=3000,
            temperature=0.5,
            top_p=0.8
        )
        print("✅ HyperClova X 초기화 성공")
    except Exception as e:
        print(f"❌ HyperClova X 초기화 실패: {e}")
        return
    
    # 테스트용 프롬프트
    prompt = '''
당신은 한국 퇴직연금 전문 분석가입니다. 아래 지시사항을 정확히 따라 답변하세요.

**중요: 다섯 가지 제목을 반드시 사용하세요. 반드시 아래 양식 그대로 답변하세요. 각 섹션의 제목을 정확히 사용하세요.**

투자자 정보: 30세 위험중립형
경제상황: GDP 3.2%, 인플레이션 2.1%, 기준금리 3.5%

---

[매크로경제분석]
한국 경제 현황과 전망을 정확히 2-3문장으로 작성하세요.

[ETF시장동향]
국내 ETF 시장 트렌드를 정확히 2-3문장으로 작성하세요.

[투자전략]
30세 위험중립형 투자자를 위한 맞춤 투자 전략을 정확히 2-3문장으로 작성하세요.

[리스크요인]
주요 위험요인을 정확히 2-3문장으로 작성하세요.

[포트폴리오]
아래 JSON 형식으로만 답변하세요. 다른 설명은 추가하지 마세요.
8개 국내 ETF의 비중 합계는 반드시 1.0이 되어야 합니다.
{"KODEX 200": 0.20, "TIGER 200": 0.15, "KODEX 국고채10년": 0.25, "KODEX 단기채권": 0.15, "KODEX 2차전지산업": 0.10, "KODEX 골드선물(H)": 0.08, "TIGER 국내리츠": 0.07}

**주의사항:**
1. 각 섹션 제목은 대괄호 [] 안에 정확히 작성
2. 각 섹션 사이에 빈 줄 삽입
3. 포트폴리오는 JSON 형식만 사용
4. 추가 설명이나 서론/결론 금지
5. 지정된 문장 수 엄수
'''
    
    try:
        # API 호출
        print("🤖 HyperClova X API 호출 중...")
        response = client.invoke(prompt)
        
        print("=" * 80)
        print("📝 HyperClova X 원본 응답:")
        print("=" * 80)
        print(response.content)
        print("=" * 80)
        
        # 응답 분석
        analyze_response(response.content)
        
        # 파싱 테스트
        test_parsing(response.content)
        
        return response.content
        
    except Exception as e:
        print(f"❌ API 호출 실패: {e}")
        return None

def analyze_response(response_text):
    """응답 분석"""
    print("\n📊 응답 분석:")
    print(f"- 총 문자 수: {len(response_text)}")
    print(f"- 줄 수: {len(response_text.split(chr(10)))}")
    
    # 백슬래시 문제 해결
    double_newline_pattern = r'\n\n+'
    paragraph_count = len(re.split(double_newline_pattern, response_text))
    print(f"- 단락 수 (두 줄바꿈 기준): {paragraph_count}")
    
    # 섹션 헤더 찾기
    sections = ['매크로경제분석', 'ETF시장동향', '투자전략', '리스크요인', '포트폴리오']
    found_sections = []
    
    for section in sections:
        if f'[{section}]' in response_text:
            found_sections.append(section)
        elif section in response_text:
            found_sections.append(f"{section} (대괄호 없음)")
    
    print(f"- 발견된 섹션: {found_sections}")
    
    # JSON 패턴 찾기
    json_pattern = r'\{[^{}]*\}'
    json_patterns = re.findall(json_pattern, response_text)
    print(f"- JSON 패턴 수: {len(json_patterns)}")
    
    if json_patterns:
        print("- 발견된 JSON:")
        for i, pattern in enumerate(json_patterns):
            print(f"  {i+1}: {pattern}")

def test_parsing(response_text):
    """파싱 테스트"""
    print("\n🔍 파싱 테스트:")
    
    # 1. 줄바꿈 기반 블록 분할
    double_newline_pattern = r"\n\n+"
    blocks = re.split(double_newline_pattern, response_text.strip())
    
    print(f"📦 분할된 블록 수: {len(blocks)}")
    
    for i, block in enumerate(blocks):
        block = block.strip()
        if len(block) > 10:
            preview = block[:100] + '...' if len(block) > 100 else block
            print(f"블록 {i+1}: {preview}")
    
    # 2. 섹션별 분류 테스트
    sections = classify_blocks_test(blocks)
    
    print("\n📋 섹션 분류 결과:")
    for section_key, content in sections.items():
        print(f"- {section_key}: {len(content)} 문자")
        if content:
            preview = content[:100] + '...' if len(content) > 100 else content
            print(f"  내용: {preview}")
    
    # 3. 포트폴리오 추출 테스트
    portfolio = extract_portfolio_test(blocks)
    
    print(f"\n💼 포트폴리오 추출 결과: {len(portfolio)}개 ETF")
    for etf_name, weight in portfolio.items():
        print(f"- {etf_name}: {weight:.3f}")

def classify_blocks_test(blocks):
    """블록 분류 테스트"""
    sections = {
        'macro': '',
        'market': '',
        'strategy': '',
        'risk': ''
    }
    
    section_patterns = {
        'macro': [r'매크로경제분석', r'경제분석', r'경제상황', r'매크로'],
        'market': [r'ETF시장동향', r'시장동향', r'ETF동향', r'시장트렌드'],
        'strategy': [r'투자전략', r'투자방향', r'투자방법', r'전략'],
        'risk': [r'리스크요인', r'위험요인', r'리스크', r'위험']
    }
    
    for block in blocks:
        block = block.strip()
        if len(block) < 10:
            continue
        
        classified = False
        for section_key, patterns in section_patterns.items():
            for pattern in patterns:
                if re.search(pattern, block, re.IGNORECASE):
                    content = extract_content_from_block_test(block, pattern)
                    if content and len(content) > 20:
                        sections[section_key] = content
                        print(f"✅ {section_key}: {len(content)} 문자 분류")
                        classified = True
                        break
            if classified:
                break
        
        if not classified:
            print(f"❓ 분류되지 않은 블록: {block[:50]}...")
    
    return sections

def extract_content_from_block_test(block, header_pattern):
    """블록에서 내용 추출 테스트"""
    bracket_pattern = rf'\[?{header_pattern}\]?'
    content = re.sub(bracket_pattern, '', block, flags=re.IGNORECASE)
    
    # 앞뒤 특수문자 제거
    content = re.sub(r'^[\[\]\s\-\*\#]+', '', content)
    content = re.sub(r'[\[\]\s]+$', '', content)
    content = re.sub(r'\s+', ' ', content)
    
    return content.strip()

def extract_portfolio_test(blocks):
    """포트폴리오 추출 테스트"""
    portfolio = {}
    
    for block in blocks:
        if '포트폴리오' in block or '{' in block:
            json_patterns = [
                r'\{[^{}]*\}',
                r'\{.*?\}',
            ]
            
            for pattern in json_patterns:
                matches = re.findall(pattern, block, re.DOTALL)
                for match in matches:
                    try:
                        cleaned_json = clean_json_test(match)
                        portfolio_data = json.loads(cleaned_json)
                        
                        if isinstance(portfolio_data, dict) and len(portfolio_data) >= 5:
                            portfolio = portfolio_data
                            print(f"✅ JSON 파싱 성공: {len(portfolio_data)}개 항목")
                            break
                    except Exception as e:
                        print(f"❌ JSON 파싱 실패: {e}")
                        print(f"   원본: {match}")
                        print(f"   정제: {clean_json_test(match)}")
                        continue
    
    return portfolio

def clean_json_test(json_str):
    """JSON 정제 테스트"""
    original = json_str
    
    # 작은따옴표를 큰따옴표로
    json_str = json_str.replace("'", '"')
    
    # 키에 따옴표 추가 (없는 경우)
    key_pattern = r'([{,]\s*)([a-zA-Z가-힣][a-zA-Z가-힣0-9\s]*)\s*:'
    json_str = re.sub(key_pattern, r'\1"\2":', json_str)
    
    # 불필요한 공백 정리
    json_str = re.sub(r'\s+', ' ', json_str)
    
    # 마지막 쉼표 제거
    json_str = re.sub(r',\s*}', '}', json_str)
    
    print(f"JSON 정제: {original} -> {json_str}")
    
    return json_str.strip()

def save_response_to_file(response_text, filename="hyperclova_response.txt"):
    """응답을 파일로 저장"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("HyperClova X 응답 (원본)\n")
            f.write("=" * 80 + "\n")
            f.write(response_text)
            f.write("\n" + "=" * 80 + "\n")
        print(f"✅ 응답이 {filename}에 저장되었습니다.")
    except Exception as e:
        print(f"❌ 파일 저장 실패: {e}")

def debug_response_structure(response_text):
    """응답 구조 디버깅"""
    print("\n🔬 응답 구조 디버깅:")
    
    # 각 줄 분석
    lines = response_text.split('\n')
    print(f"총 {len(lines)}줄:")
    
    for i, line in enumerate(lines[:20]):  # 처음 20줄만 표시
        line_repr = repr(line)
        print(f"  {i+1:2d}: {line_repr}")
    
    if len(lines) > 20:
        print(f"  ... (나머지 {len(lines)-20}줄 생략)")
    
    # 빈 줄 위치 확인
    empty_lines = [i+1 for i, line in enumerate(lines) if line.strip() == '']
    print(f"\n빈 줄 위치: {empty_lines}")

if __name__ == "__main__":
    print("🧪 HyperClova X 응답 테스트 시작")
    print("=" * 80)
    
    response = test_hyperclova_response()
    
    if response:
        # 구조 디버깅
        debug_response_structure(response)
        
        # 파일로 저장
        save_response_to_file(response)
        
        print("\n" + "=" * 80)
        print("🎯 테스트 완료!")
        print("응답을 확인하여 파싱 로직을 개선하세요.")
        print("hyperclova_response.txt 파일도 확인해보세요.")
    else:
        print("❌ 테스트 실패")


🧪 HyperClova X 응답 테스트 시작
✅ HyperClova X 초기화 성공
🤖 HyperClova X API 호출 중...
📝 HyperClova X 원본 응답:
[매크로경제분석]  
현재 한국의 GDP는 3.2% 성장 중이며, 인플레이션율은 2.1%, 기준금리는 3.5%를 유지하고 있습니다. 이러한 경제 상황 속에서 한국은 안정적인 성장을 지속할 것으로 보이지만, 글로벌 경기 둔화와 같은 외부 요인에 따른 리스크도 존재합니다. 따라서 투자자들은 국내외 경제 동향을 주의 깊게 살펴보며 신중한 투자 결정을 내려야 할 것입니다.

[ETF시장동향]  
최근 국내 ETF 시장은 다양한 테마와 자산군에 대한 투자자들의 관심이 증가하고 있습니다. 특히 ESG(환경·사회·지배구조) 및 기술 관련 ETF들이 주목받고 있으며, 채권 및 부동산 등 안전자산에 대한 수요 또한 꾸준히 이어지고 있습니다. 이에 따라 국내 ETF 시장의 다변화가 가속화되고 있는 추세입니다.

[투자전략]  
30세 위험중립형 투자자에게는 균형 잡힌 포트폴리오 구성이 중요합니다. 주식과 채권의 비율을 조절하여 변동성에 대비하고, 특정 산업보다는 분산투자를 통해 리스크를 최소화하는 것이 좋습니다. 또한 장기적인 관점에서 ESG와 같은 테마를 고려하며 투자를 진행하는 것도 추천됩니다.

[리스크요인]  
한국 경제에서 주요 위험 요소로는 글로벌 공급망 이슈, 원자재 가격 상승, 그리고 금리 변동이 있습니다. 특히 미국 연준(Fed)의 금리 정책 변화는 국내 금융 시장에 큰 영향을 미칠 수 있으므로 지속적인 모니터링이 필요합니다. 더불어 인구 고령화 문제 역시 장기적으로 경제 성장에 부정적인 영향을 줄 가능성이 높습니다.

[포트폴리오]
```json
{
    "KODEX 200": 0.20,
    "TIGER 200": 0.15,
    "KODEX 국고채10년": 0.25,
    "KODEX 단기채권": 0.15,
    "KODEX 2차전지산업": 0.10,
    "KODEX 골드선물

In [1]:
# test_macro_and_ai_analysis.py
import os
import sys
sys.path.append('.')

import pandas as pd
import json
from datetime import datetime
from data_collector import DataCollector
from ai_analyzer import AIAnalyzer
from config import Config

def test_macro_data_collection():
    """매크로 경제 데이터 수집 테스트"""
    print("=" * 80)
    print("🔍 매크로 경제 데이터 수집 테스트")
    print("=" * 80)
    
    data_collector = DataCollector()
    
    # 1. 경제지표 수집 테스트
    print("\n📊 경제지표 데이터 수집 중...")
    economic_data = data_collector.get_economic_indicators()
    
    print(f"📈 수집된 경제지표 수: {len(economic_data)}")
    
    # 각 지표별 상세 정보 출력
    for indicator_name, data in economic_data.items():
        print(f"\n--- {indicator_name} ---")
        print(f"  현재값: {data.get('current', 'N/A')}")
        print(f"  이전값: {data.get('previous', 'N/A')}")
        print(f"  추세: {data.get('trend', 'N/A')}")
        print(f"  단위: {data.get('unit', 'N/A')}")
        print(f"  마지막 업데이트: {data.get('last_update', 'N/A')}")
        print(f"  데이터 개수: {data.get('data_count', 'N/A')}")
        print(f"  샘플 데이터 여부: {data.get('is_sample', False)}")
        
        # 히스토리 데이터 확인
        history = data.get('history', [])
        if history and len(history) > 0:
            print(f"  최근 5개 값: {history[-5:]}")
    
    # 2. 매크로 데이터 포맷팅 테스트
    print("\n🔧 매크로 데이터 포맷팅 테스트")
    ai_analyzer = AIAnalyzer()
    formatted_macro = ai_analyzer._format_macro_data(economic_data)
    print(f"포맷된 매크로 데이터: {formatted_macro}")
    
    return economic_data

def test_ai_market_analysis_vs_portfolio():
    """AI 시장분석과 포트폴리오 추천 결과 비교 테스트"""
    print("\n" + "=" * 80)
    print("🤖 AI 시장분석 vs 포트폴리오 추천 비교 테스트")
    print("=" * 80)
    
    # 테스트용 데이터 준비
    sample_macro_data = {
        'GDP': {'current': 3.4, 'previous': 3.2, 'trend': 'up', 'unit': '%'},
        'CPI': {'current': 2.3, 'previous': 2.1, 'trend': 'up', 'unit': '%'},
        'INTEREST_RATE': {'current': 3.25, 'previous': 3.5, 'trend': 'down', 'unit': '%'}
    }
    
    sample_etf_data = {
        '국내주식형': {
            'KODEX 200': {'code': '069500', 'price': 12000, 'category': '국내주식형'},
            'TIGER 200': {'code': '102110', 'price': 11800, 'category': '국내주식형'}
        },
        '국내채권형': {
            'KODEX 국고채10년': {'code': '148070', 'price': 105000, 'category': '국내채권형'},
            'KODEX 단기채권': {'code': '130730', 'price': 102000, 'category': '국내채권형'}
        },
        '국내섹터/테마': {
            'KODEX 2차전지산업': {'code': '117700', 'price': 8500, 'category': '국내섹터/테마'},
            'KODEX 바이오': {'code': '139660', 'price': 7200, 'category': '국내섹터/테마'}
        },
        '국내대안투자': {
            'KODEX 골드선물(H)': {'code': '132030', 'price': 55000, 'category': '국내대안투자'},
            'TIGER 국내리츠': {'code': '143460', 'price': 9800, 'category': '국내대안투자'}
        }
    }
    
    sample_user_profile = {
        'age': 35,
        'risk_tolerance': '위험중립형',
        'investment_period': 20,
        'current_assets': 50000000,
        'monthly_contribution': 1000000
    }
    
    # AI 분석기 초기화
    ai_analyzer = AIAnalyzer()
    
    # 1. AI 종합 분석 실행
    print("\n🔍 AI 종합 분석 실행 중...")
    comprehensive_result = ai_analyzer.comprehensive_market_analysis(
        sample_macro_data, sample_etf_data, sample_user_profile
    )
    
    # 2. 결과 분석 및 출력
    if comprehensive_result:
        print("✅ AI 종합 분석 성공")
        
        # 시장 분석 결과
        analysis = comprehensive_result.get('analysis', {})
        print("\n📊 AI 시장 분석 결과:")
        for section_name, content in analysis.items():
            print(f"  {section_name}: {len(content)} 문자")
            print(f"    내용: {content[:100]}{'...' if len(content) > 100 else ''}")
        
        # 포트폴리오 결과
        portfolio = comprehensive_result.get('portfolio', {})
        weights = portfolio.get('weights', {})
        
        print(f"\n💼 AI 포트폴리오 결과:")
        print(f"  ETF 수: {len(weights)}")
        print(f"  총 가중치 합: {sum(weights.values()):.3f}")
        
        print("\n  개별 ETF 비중:")
        for etf_name, weight in weights.items():
            print(f"    {etf_name}: {weight*100:.1f}%")
        
        print(f"\n  배분 근거: {portfolio.get('allocation_reasoning', 'N/A')}")
        print(f"  분산 전략: {portfolio.get('diversification_strategy', 'N/A')}")
        print(f"  데이터 소스: {comprehensive_result.get('source', 'N/A')}")
        
        # 카테고리별 분포
        category_dist = portfolio.get('category_distribution', {})
        if category_dist:
            print("\n📈 카테고리별 분포:")
            category_weights = category_dist.get('category_weights', {})
            for category, weight in category_weights.items():
                if weight > 0:
                    print(f"    {category}: {weight*100:.1f}%")
        
        return comprehensive_result
    else:
        print("❌ AI 종합 분석 실패")
        return None

def test_ai_response_consistency():
    """AI 응답 일관성 테스트 (동일 입력에 대한 여러 번 실행)"""
    print("\n" + "=" * 80)
    print("🔄 AI 응답 일관성 테스트")
    print("=" * 80)
    
    # 동일한 입력 데이터
    test_macro_data = {
        'GDP': {'current': 3.2, 'previous': 3.0, 'trend': 'up'},
        'CPI': {'current': 2.1, 'previous': 2.0, 'trend': 'up'},
        'INTEREST_RATE': {'current': 3.5, 'previous': 3.5, 'trend': 'stable'}
    }
    
    test_etf_data = {
        '국내주식형': {'KODEX 200': {'code': '069500', 'price': 12000}},
        '국내채권형': {'KODEX 국고채10년': {'code': '148070', 'price': 105000}}
    }
    
    test_user_profile = {
        'age': 40,
        'risk_tolerance': '위험중립형',
        'investment_period': 15
    }
    
    ai_analyzer = AIAnalyzer()
    results = []
    
    # 3번 실행하여 일관성 확인
    for i in range(3):
        print(f"\n🔄 테스트 실행 {i+1}/3")
        result = ai_analyzer.comprehensive_market_analysis(
            test_macro_data, test_etf_data, test_user_profile
        )
        results.append(result)
        
        if result:
            analysis = result.get('analysis', {})
            portfolio = result.get('portfolio', {})
            weights = portfolio.get('weights', {})
            
            print(f"  분석 섹션 수: {len(analysis)}")
            print(f"  포트폴리오 ETF 수: {len(weights)}")
            print(f"  가중치 합: {sum(weights.values()):.3f}")
            print(f"  데이터 소스: {result.get('source', 'N/A')}")
        else:
            print("  ❌ 분석 실패")
    
    # 일관성 분석
    print("\n📊 일관성 분석:")
    successful_results = [r for r in results if r is not None]
    
    if len(successful_results) > 1:
        # 분석 내용 길이 비교
        analysis_lengths = []
        for result in successful_results:
            analysis = result.get('analysis', {})
            total_length = sum(len(content) for content in analysis.values())
            analysis_lengths.append(total_length)
        
        print(f"  분석 내용 길이 범위: {min(analysis_lengths)} ~ {max(analysis_lengths)} 문자")
        
        # 포트폴리오 ETF 수 비교
        etf_counts = []
        for result in successful_results:
            weights = result.get('portfolio', {}).get('weights', {})
            etf_counts.append(len(weights))
        
        print(f"  포트폴리오 ETF 수 범위: {min(etf_counts)} ~ {max(etf_counts)}개")
        
        # 일관성 점수 계산
        length_consistency = 1 - (max(analysis_lengths) - min(analysis_lengths)) / max(analysis_lengths) if max(analysis_lengths) > 0 else 1
        etf_consistency = 1 - (max(etf_counts) - min(etf_counts)) / max(etf_counts) if max(etf_counts) > 0 else 1
        
        overall_consistency = (length_consistency + etf_consistency) / 2
        print(f"  전체 일관성 점수: {overall_consistency:.2f} (1.0이 완전 일관성)")
    
    return results

def test_data_flow_integration():
    """데이터 흐름 통합 테스트"""
    print("\n" + "=" * 80)
    print("🔗 데이터 흐름 통합 테스트")
    print("=" * 80)
    
    try:
        # 1. 데이터 수집
        print("1️⃣ 데이터 수집 단계")
        data_collector = DataCollector()
        
        print("  📊 경제지표 수집...")
        economic_data = data_collector.get_economic_indicators()
        print(f"  ✅ 경제지표 {len(economic_data)}개 수집 완료")
        
        print("  📈 ETF 데이터 수집...")
        etf_data = data_collector.get_etf_data(period='3m')  # 3개월 데이터
        etf_count = sum(len(etfs) for etfs in etf_data.values())
        print(f"  ✅ ETF {etf_count}개 수집 완료")
        
        print("  📉 시장 데이터 수집...")
        market_data = data_collector.get_market_data()
        print(f"  ✅ 시장 데이터 수집 완료")
        
        # 2. AI 분석
        print("\n2️⃣ AI 분석 단계")
        ai_analyzer = AIAnalyzer()
        
        user_profile = {
            'age': 35,
            'risk_tolerance': '위험중립형',
            'investment_period': 20,
            'current_assets': 30000000,
            'monthly_contribution': 500000
        }
        
        print("  🤖 AI 종합 분석 실행...")
        ai_result = ai_analyzer.comprehensive_market_analysis(
            economic_data, etf_data, user_profile
        )
        
        if ai_result:
            print("  ✅ AI 분석 성공")
            
            # 3. 결과 검증
            print("\n3️⃣ 결과 검증 단계")
            
            # 분석 섹션 검증
            analysis = ai_result.get('analysis', {})
            required_sections = ['macro_analysis', 'market_trends', 'investment_strategy', 'risk_factors']
            
            for section in required_sections:
                content = analysis.get(section, '')
                status = "✅" if len(content) > 20 else "❌"
                print(f"    {status} {section}: {len(content)} 문자")
            
            # 포트폴리오 검증
            portfolio = ai_result.get('portfolio', {})
            weights = portfolio.get('weights', {})
            
            total_weight = sum(weights.values())
            weight_status = "✅" if abs(total_weight - 1.0) < 0.1 else "❌"
            print(f"    {weight_status} 포트폴리오 가중치 합: {total_weight:.3f}")
            
            etf_count_status = "✅" if len(weights) >= 5 else "❌"
            print(f"    {etf_count_status} ETF 개수: {len(weights)}개")
            
            # 4. 성과 계산 테스트
            print("\n4️⃣ 성과 계산 단계")
            from utils import calculate_portfolio_performance
            
            print("  📊 포트폴리오 성과 계산...")
            performance = calculate_portfolio_performance(weights, etf_data)
            
            if performance:
                print(f"  ✅ 연환산 수익률: {performance['expected_return']*100:.2f}%")
                print(f"  ✅ 연환산 변동성: {performance['volatility']*100:.2f}%")
                print(f"  ✅ 샤프 비율: {performance['sharpe_ratio']:.3f}")
                print(f"  ✅ 최대 낙폭: {performance['max_drawdown']*100:.2f}%")
            else:
                print("  ❌ 성과 계산 실패")
            
            print("\n🎯 통합 테스트 완료!")
            return True
            
        else:
            print("  ❌ AI 분석 실패")
            return False
            
    except Exception as e:
        print(f"❌ 통합 테스트 중 오류 발생: {e}")
        return False

def save_test_results(results, filename="test_results.json"):
    """테스트 결과를 파일로 저장"""
    try:
        # 결과를 JSON 직렬화 가능한 형태로 변환
        serializable_results = {}
        
        for key, value in results.items():
            if isinstance(value, dict):
                serializable_results[key] = value
            else:
                serializable_results[key] = str(value)
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(serializable_results, f, ensure_ascii=False, indent=2)
        
        print(f"✅ 테스트 결과가 {filename}에 저장되었습니다.")
        
    except Exception as e:
        print(f"❌ 결과 저장 실패: {e}")

if __name__ == "__main__":
    print("🧪 매크로 데이터 및 AI 분석 종합 테스트 시작")
    print(f"📅 테스트 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    test_results = {}
    
    # 1. 매크로 데이터 수집 테스트
    try:
        macro_data = test_macro_data_collection()
        test_results['macro_data_test'] = 'success'
        test_results['macro_data_count'] = len(macro_data)
    except Exception as e:
        print(f"❌ 매크로 데이터 테스트 실패: {e}")
        test_results['macro_data_test'] = f'failed: {e}'
    
    # 2. AI 분석 비교 테스트
    try:
        ai_result = test_ai_market_analysis_vs_portfolio()
        test_results['ai_analysis_test'] = 'success' if ai_result else 'failed'
        if ai_result:
            test_results['ai_etf_count'] = len(ai_result.get('portfolio', {}).get('weights', {}))
            test_results['ai_source'] = ai_result.get('source', 'unknown')
    except Exception as e:
        print(f"❌ AI 분석 테스트 실패: {e}")
        test_results['ai_analysis_test'] = f'failed: {e}'
    
    # 3. AI 일관성 테스트
    try:
        consistency_results = test_ai_response_consistency()
        successful_count = len([r for r in consistency_results if r is not None])
        test_results['consistency_test'] = f'{successful_count}/3 successful'
    except Exception as e:
        print(f"❌ 일관성 테스트 실패: {e}")
        test_results['consistency_test'] = f'failed: {e}'
    
    # 4. 통합 테스트
    try:
        integration_success = test_data_flow_integration()
        test_results['integration_test'] = 'success' if integration_success else 'failed'
    except Exception as e:
        print(f"❌ 통합 테스트 실패: {e}")
        test_results['integration_test'] = f'failed: {e}'
    
    # 결과 저장
    test_results['test_timestamp'] = datetime.now().isoformat()
    save_test_results(test_results)
    
    print("\n" + "=" * 80)
    print("🎯 전체 테스트 완료!")
    print("=" * 80)
    
    # 요약 출력
    for test_name, result in test_results.items():
        if test_name != 'test_timestamp':
            print(f"  {test_name}: {result}")


🧪 매크로 데이터 및 AI 분석 종합 테스트 시작
📅 테스트 일시: 2025-06-19 11:31:44
🔍 매크로 경제 데이터 수집 테스트

📊 경제지표 데이터 수집 중...
📊 경제지표 수집 기간: 202306 ~ 202506
📈 경제지표 수집 중: GDP (200Y001)
⚠️ GDP: API 응답 구조 오류
📈 경제지표 수집 중: CPI (901Y009)
✅ CPI: 100.65 (최신: 202306)
📈 경제지표 수집 중: PPI (404Y014)
✅ PPI: 108.32 (최신: 202306)
📈 경제지표 수집 중: INTEREST_RATE (722Y001)
✅ INTEREST_RATE: 3.5 (최신: 202405)
📈 경제지표 수집 중: EXCHANGE_RATE (731Y001)
⚠️ EXCHANGE_RATE: API 응답 구조 오류
📈 경제지표 수집 중: MONEY_SUPPLY (101Y002)
✅ MONEY_SUPPLY: 712665.0 (최신: 202312)
📈 경제지표 수집 중: EMPLOYMENT (101Y003)
✅ EMPLOYMENT: 3986786.5 (최신: 202403)
📈 경제지표 수집 중: INDUSTRIAL_PRODUCTION (403Y001)
✅ INDUSTRIAL_PRODUCTION: 123.72 (최신: 202306)
🎯 경제지표 수집 완료: 8개
📈 수집된 경제지표 수: 8

--- GDP ---
  현재값: 3.0617692115169834
  이전값: 3.0797630103882407
  추세: down
  단위: %
  마지막 업데이트: 202506
  데이터 개수: 24
  샘플 데이터 여부: True
  최근 5개 값: [3.3483298361668217, 3.106156221187253, 3.1476933265157307, 3.0208005955484025, 3.146324814489925]

--- CPI ---
  현재값: 100.65
  이전값: 90.2
  추세: up
  단위: 2020=100
  